# NER and Sentiment

In this section we will work through applying basic sentiment analysis to our data using a pre-built *distilBERT* model from the **Flair** library. We will then use our organization labels captured through NER in the previous section to create a list of organizations with the highest and lowest average sentiment scores.

In [1]:
import pandas as pd
import flair

We initialize the English sentiment model `en-sentiment`:

In [3]:
model = flair.models.TextClassifier.load('en-sentiment')

2022-08-04 20:31:08,008 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


For each sample there are a few steps we need to take to create the sentiment score. We need to tokenize the input text, make a prediction, extract the direction (*positive* or *negative*) and confidence (a score from 0 to 1). If this is new to you, we cover the Flair sentiment model in more depth in **TK insert link**.

The following function carries out each of these steps for a single extract:

In [4]:
def get_sentiment(text):
    # tokenize input text
    sentence = flair.data.Sentence(text)
    # make sentiment prediction
    model.predict(sentence)
    # extract sentiment direction and confidence (label and score) object
    sentiment = sentence.labels[0]
    return sentiment

We now need to load our previously processed dataframe (which includes the *organizations* column) and `apply` the `get_sentiment` function to the *selftext* column. These sentiment scores will then be stored in a new *sentiment* column.

In [5]:
# load data
df = pd.read_csv('./data/processed/reddit_investing_ner.csv', sep='|')
df.head()

,id,created_utc,subreddit,title,selftext,upvote_ratio,ups,downs,score,organizations
0,t3_wg6ntw,1.659633e+09,investing,Is this the end of the traditional stock market?,Please consider the following points from a pe...,0.37,0.0,0.0,0.0,[]
1,t3_wg6bl9,1.659632e+09,investing,NERV is going nuts. Are my options worthless?,What happens to options when they are no longe...,0.50,0.0,0.0,0.0,"['Minerva Neurosciences', 'NERV']"
2,t3_wg29p4,1.659622e+09,investing,Earnings boost Toronto stocks,[https://www.reuters.com/markets/stocks/tsx-fu...,0.81,82.0,0.0,82.0,"['Canadian Natural Resources', ""the Toronto St..."
3,t3_wg0os3,1.659618e+09,investing,Convertible bond and nuke,Can someone please explain to me what this “nu...,0.85,41.0,0.0,41.0,[]
4,t3_wfw9yh,1.659604e+09,investing,Daily General Discussion and Advice Thread - A...,Have a general question? Want to offer some c...,0.86,22.0,0.0,22.0,[]


In [6]:
# get sentiment
df['sentiment'] = df['selftext'].apply(get_sentiment)
df.head()

,id,created_utc,subreddit,title,selftext,upvote_ratio,ups,downs,score,organizations,sentiment
0,t3_wg6ntw,1.659633e+09,investing,Is this the end of the traditional stock market?,Please consider the following points from a pe...,0.37,0.0,0.0,0.0,[],NEGATIVE (0.9999)
1,t3_wg6bl9,1.659632e+09,investing,NERV is going nuts. Are my options worthless?,What happens to options when they are no longe...,0.50,0.0,0.0,0.0,"['Minerva Neurosciences', 'NERV']",NEGATIVE (0.9989)
2,t3_wg29p4,1.659622e+09,investing,Earnings boost Toronto stocks,[https://www.reuters.com/markets/stocks/tsx-fu...,0.81,82.0,0.0,82.0,"['Canadian Natural Resources', ""the Toronto St...",POSITIVE (0.9677)
3,t3_wg0os3,1.659618e+09,investing,Convertible bond and nuke,Can someone please explain to me what this “nu...,0.85,41.0,0.0,41.0,[],NEGATIVE (0.9979)
4,t3_wfw9yh,1.659604e+09,investing,Daily General Discussion and Advice Thread - A...,Have a general question? Want to offer some c...,0.86,22.0,0.0,22.0,[],NEGATIVE (0.9993)


Now we need to extract each of the organizations alongside it's sentiment score. We will then loop through each, tallying up a total sentiment score and count.

Before we do that, we need to convert each value in the *organizations* column to a list (they are currently strings because we cannot save Python lists to file within Pandas dataframes, they are automatically converted to strings).

In [7]:
import ast

df['organizations'] = df['organizations'].apply(lambda x: ast.literal_eval(x))

In [10]:
# initialize sentiment dictionary
sentiment = {}

# loop through dataframe and extract org labels and sentiment scores into sentiment dictionary
for i, row in df.iterrows():
    # extract sentiment direction and score
    direction = row['sentiment'].value
    score = row['sentiment'].score
    # loop through each label in organizations column
    for org in row['organizations']:
        # check if org label exists in sentiment dictionary already
        if org not in sentiment.keys():
            # if it doesn't, initialize new entry in dictionary
            sentiment[org] = {'POSITIVE': [], 'NEGATIVE': []}
        # append positive/negative score to respective dictionary entry
        sentiment[org][direction].append(score)

In [12]:
sentiment

{'Minerva Neurosciences': {'POSITIVE': [], 'NEGATIVE': [0.9988642930984497]},
 'NERV': {'POSITIVE': [], 'NEGATIVE': [0.9988642930984497]},
 'Canadian Natural Resources': {'POSITIVE': [0.967677891254425],
  'NEGATIVE': []},
 "the Toronto Stock Exchange's": {'POSITIVE': [0.967677891254425],
  'NEGATIVE': []},
 'Bombardier': {'POSITIVE': [0.967677891254425], 'NEGATIVE': []},
 '.GSPTSE)**](https://www.reuters.com/quote/.GSPTSE': {'POSITIVE': [0.967677891254425],
  'NEGATIVE': []},
 'Reuters': {'POSITIVE': [0.967677891254425,
   0.9876999855041504,
   0.928656280040741],
  'NEGATIVE': [0.9999175071716309,
   0.9997807145118713,
   0.9997915625572205,
   0.9999701976776123,
   0.9715436697006226,
   0.9947078227996826,
   0.9875293970108032,
   0.9921545386314392,
   0.9998914003372192,
   0.9739415049552917,
   0.9999747276306152,
   0.9999727010726929,
   0.9523166418075562,
   0.9999068975448608,
   0.9650932550430298,
   0.9999666213989258]},
 'EPS': {'POSITIVE': [0.5656468868255615, 0.9

Now we can loop through each organization entry in the sentiment dictionary and calculate an average positive, and average negative score:

In [13]:
# initialize sentiment list
avg_sentiment = []

# loop through each organization
for org in sentiment.keys():
    # get number of positive and negative ratings
    freq = len(sentiment[org]['POSITIVE']) + len(sentiment[org]['NEGATIVE'])
    for direction in ['POSITIVE', 'NEGATIVE']:
        # assign to variable for cleaner code
        score = sentiment[org][direction]
        # if there are no entries, set to 0
        if len(score) == 0:
            sentiment[org][direction] = 0.0
        else:
            # otherwise calculate total
            sentiment[org][direction] = sum(score)
    # now calculate total amount
    total = sentiment[org]['POSITIVE'] - sentiment[org]['NEGATIVE']
    # and the average score
    avg = total/freq
    # add to sentiment list
    avg_sentiment.append({
        'entity': org,
        'positive': sentiment[org]['POSITIVE'],
        'negative': sentiment[org]['NEGATIVE'],
        'frequency': freq,
        'score': avg
    })

In [30]:
sentiment_df = pd.DataFrame(avg_sentiment)
sentiment_df.head(20)

,entity,positive,negative,frequency,score
0,Minerva Neurosciences,0.000000,0.998864,1,-0.998864
1,NERV,0.000000,0.998864,1,-0.998864
2,Canadian Natural Resources,0.967678,0.000000,1,0.967678
3,the Toronto Stock Exchange's,0.967678,0.000000,1,0.967678
4,Bombardier,0.967678,0.000000,1,0.967678
5,.GSPTSE)**](https://www.reuters.com/quote/.GSPTSE,0.967678,0.000000,1,0.967678
6,Reuters,2.884034,15.836459,19,-0.681707
7,EPS,1.561494,9.595955,12,-0.669538
8,LOC,0.000000,0.997452,1,-0.997452
9,HELOC,0.000000,1.975640,2,-0.987820


Immediately we can see we have a lot of entities which have appeared once in our dataset, and because of this their score will be pushed to one extreme or the other. We can filter out anything with less than or equal to a frequency of `3` to remove many of these instances:

In [32]:
sentiment_df = sentiment_df[sentiment_df['frequency'] > 3]
sentiment_df

,entity,positive,negative,frequency,score
6,Reuters,2.884034,15.836459,19,-0.681707
7,EPS,1.561494,9.595955,12,-0.669538
10,LLC,0.739017,4.698678,6,-0.659943
16,Intel,0.565647,2.985005,4,-0.604839
18,ESG,0.000000,4.922618,5,-0.984524
24,USD,0.525487,9.685458,11,-0.832725
55,Vanguard,0.000000,10.522471,11,-0.956588
56,Amazon,3.989256,6.282721,12,-0.191122
78,max,0.628914,5.937832,7,-0.758417
89,SPY,0.000000,3.998682,4,-0.999671


Here we have some more relevant information. We can see a few items that we can remove through the `BLACKLIST` covered in earlier sections such as *Fed* and *Treasury*, but nonetheless this list is looking much better than before. We can apply `sort` to search for the entities with the highest overall score:

In [33]:
sentiment_df.sort_values('score', ascending=False).head(10)

,entity,positive,negative,frequency,score
99,Meta,2.375666,2.999957,6,-0.104048
56,Amazon,3.989256,6.282721,12,-0.191122
187,Disney,1.530301,2.651182,5,-0.224176
169,PCE,0.840871,2.742924,4,-0.475513
256,FOMC,0.786323,2.976080,4,-0.547439
260,NASDAQ,1.352008,6.681146,9,-0.592126
16,Intel,0.565647,2.985005,4,-0.604839
177,Tesla,1.827942,8.599928,11,-0.615635
10,LLC,0.739017,4.698678,6,-0.659943
373,QQQ,0.966242,4.980826,6,-0.669097


Very quickly we've got our results that we have pulled together using simple, ready-to-use models, and **zero** text preprocessing. With further fine-tuning, and process development, these already good results can become great. Which we will cover soon.